<a href="https://colab.research.google.com/github/sulphatet/Archives/blob/main/activation_init.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib as plt

In [ ]:
D = np.random.randn(1000,500)
#print(len(D[0,]))
hidden_layer_size = [500]*10
#print(hidden_layer_size)
nonlinearities = ['tanh']*len(hidden_layer_size)

In [ ]:
act = {'relu':lambda x:np.maximum(0,x),'tanh':lambda x:np.tanh(x)}
Hs = {}
from collections import defaultdict
X = defaultdict(list)

In [ ]:
for i in range(len(hidden_layer_size)):
  X = D if i == 0 else Hs[i-1]
  fan_in = X.shape[1]
  fan_out = hidden_layer_size[i]
  #print(fan_in)
  #print(fan_out)
  W = np.random.randn(fan_in,fan_out)*0.01
  H = np.dot(X,W)
  H = act[nonlinearities[i]](H)
  Hs[i] = H

In [1]:
import torch
from torch.autograd import Variable

dtype = torch.cuda.FloatTensor
N,D_in,D_out,H = 64,1000,10,100

x = torch.randn(N,D_in).type(dtype)
y = torch.randn(N,D_out).type(dtype)
w1 = torch.randn(D_in,H).type(dtype)
w2 = torch.randn(H,D_out).type(dtype)



learning_rate = 1e-6
for t in range(500):
  h = x.mm(w1)
  h_relu = h.clamp(min = 0)
  y_pred = h_relu.mm(w2)
  loss = (y_pred-y).pow(2).sum()

  grad_y_pred = 2.0*(y_pred-y)
  grad_w2 = h_relu.t().mm(grad_y_pred)
  grad_h_relu = grad_y_pred.mm(w2.t())
  grad_h = grad_h_relu.clone()
  grad_h[h<0] = 0
  grad_w1 = x.t().mm(grad_h)


  w1 -= learning_rate * grad_w1
  w2 -= learning_rate * grad_w2



In [5]:
import torch
from torch.autograd import Variable
class ReLU:
  pass


N,D_in,D_out,H = 64,1000,10,100

x = Variable(torch.randn(N,D_in),requires_grad = False)
y = Variable(torch.randn(N,D_out), requires_grad = False)
w1 = Variable(torch.randn(D_in,H),requires_grad = True)
w2 = Variable(torch.randn(H,D_out),requires_grad = True)



learning_rate = 1e-6
for t in range(500):
  relu = ReLU()
  y_pred = relu(x.mm(w1)).mm(w2)
  loss = torch.nn.MSELoss(size_average = False)

  if w1.grad: w1.grad.data.zero_()
  if w2.grad: w2.grad.data.zero_()

  loss.backward()


  w1.data -= learning_rate * w1.grad.data
  w2.data -= learning_rate * w2.grad.data



TypeError: ignored

In [3]:
import torch
from torch.autograd import Variable

N,D_in,D_out,H = 64,1000,10,100

x = Variable(torch.randn(N,D_in))
y = Variable(torch.randn(N,D_out), requires_grad = False)

model = torch.nn.Sequential(
    torch.nn.Linear(D_in,H),
    torch.nn.ReLU(),
    torch.nn.Linear(H,D_out)
)
loss_fn = torch.nn.MSELoss(size_average = False)


learning_rate = 1e-4
for t in range(500):
  y_pred = model(x)
  loss = loss_fn(y_pred,y)
  
  model.zero_grad()
  loss.backward()

  for param in model.parameters():
    param.data -= learning_rate*param.grad.data
    

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Parameter containing:
tensor([[ 0.0297, -0.0235, -0.0055,  ...,  0.0067,  0.0107, -0.0172],
        [ 0.0272,  0.0055, -0.0180,  ...,  0.0009,  0.0297, -0.0258],
        [-0.0177, -0.0001, -0.0198,  ..., -0.0260,  0.0074,  0.0212],
        ...,
        [-0.0285,  0.0293,  0.0203,  ...,  0.0141, -0.0056, -0.0217],
        [-0.0012, -0.0029, -0.0257,  ...,  0.0211, -0.0101, -0.0014],
        [ 0.0292,  0.0088,  0.0054,  ...,  0.0072,  0.0202, -0.0022]],
       requires_grad=True)


In [4]:
import torch
from torch.autograd import Variable

N,D_in,D_out,H = 64,1000,10,100

x = Variable(torch.randn(N,D_in))
y = Variable(torch.randn(N,D_out), requires_grad = False)

model = torch.nn.Sequential(
    torch.nn.Linear(D_in,H),
    torch.nn.ReLU(),
    torch.nn.Linear(H,D_out)
)
loss_fn = torch.nn.MSELoss(size_average = False)


learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)
for t in range(500):
  y_pred = model(x)
  loss = loss_fn(y_pred,y)
  
  model.zero_grad()
  loss.backward()

  optimizer.step()
    

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [6]:
import torch
from torch.autograd import Variable

class TwoLayerNetwork(torch.nn.Module):
  def __init__(self,D_in,H,D_out):
    super(TwoLayerNetwork,self).__init__()
    self.linear1 = torch.nn.Linear(D_in,H)
    self.linear2 = torch.nn.Linear(H,D_out)

  def forward(self,x):
    h_relu = self.linear1(x).clamp(min = 0)
    y_pred = self.linear2(h_relu)
    return y_pred

N,D_in,D_out,H = 64,1000,10,100

x = Variable(torch.randn(N,D_in))
y = Variable(torch.randn(N,D_out), requires_grad = False)

model = TwoLayerNetwork(D_in,H,D_out)

criterion = torch.nn.MSELoss(size_average = False)
optimizer = torch.optim.SGD(model.parameters(),lr = 1e-5)

for t in range(500):
  y_pred = model(x)
  loss = criterion(y_pred,y)

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [1]:
import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset


class TwoLayerNetwork(torch.nn.Module):
  def __init__(self,D_in,H,D_out):
    super(TwoLayerNetwork,self).__init__()
    self.linear1 = torch.nn.Linear(D_in,H)
    self.linear2 = torch.nn.Linear(H,D_out)

  def forward(self,x):
    h_relu = self.linear1(x).clamp(min = 0)
    y_pred = self.linear2(h_relu)
    return y_pred

N,D_in,D_out,H = 64,1000,10,100

x = torch.randn(N,D_in)
y = torch.randn(N,D_out)

loader = DataLoader(TensorDataset(x,y),batch_size = 8)

model = TwoLayerNetwork(D_in,H,D_out)

criterion = torch.nn.MSELoss(size_average = False)
optimizer = torch.optim.SGD(model.parameters(),lr = 1e-5)

for epoch in range(10):
  for x_batch,y_batch in loader:
    x_var,y_var = Variable(x),Variable(y)
    y_pred = model(x)
    loss = criterion(y_pred,y_var)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [2]:
import torch
import torchvision

alexnet = torchvision.models.alexnet(pretrained = True)
vgg16 = torchvision.models.vgg16(pretrained = True)
resnet101 = torchvision.models.resnet101(pretrained = True)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


  0%|          | 0.00/171M [00:00<?, ?B/s]